In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import time
from pprint import pprint
import requests
from datetime import date, timedelta, datetime
import json
from pprint import pprint
from tqdm import tqdm
from tqdm import tqdm_notebook
# sqlite Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine
# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session

In [2]:
from nba_api.stats.endpoints import playercareerstats, drafthistory, commonplayerinfo, playerawards

In [ ]:
# Not used
# cumestatsplayer,draftcombinedrillresults,playerdashboardbyteamperformance, leagueleaders, leaguedashplayerstats, draftcombinenonstationaryshooting, draftcombinestats, commonallplayers,

In [3]:
headers = {'Host': 'stats.nba.com', 
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
           'Accept': 'application/json, text/plain, */*',
           'Accept-Language': 'en-US,en;q=0.5',
           'Accept-Encoding': 'gzip, deflate, br',
           'x-nba-stats-origin': 'stats',
           'x-nba-stats-token': 'true',
           'Connection': 'keep-alive',
           'Referer': 'https://stats.nba.com/',
           'Pragma': 'no-cache',
           'Cache-Control': 'no-cache'}

In [ ]:
# pd.show_versions()

## Static Examples

In [87]:
from nba_api.stats.static import players
# Find players by full name.
players.find_players_by_full_name('Stromile Swift')

# # Find players by first name.
# players.find_players_by_first_name('lebron')

# # Find players by last name.
# players.find_players_by_last_name('^(james|love)$')

# Get all players.
# players.get_players()

[{'id': 2031,
  'full_name': 'Stromile Swift',
  'first_name': 'Stromile',
  'last_name': 'Swift',
  'is_active': False}]

## Begin Database Connection (sqlite)

In [ ]:
# Base = declarative_base()
# engine = create_engine('sqlite:///db.sqlite', echo=False)
# conn = engine.connect()
# Create (if not already in existence) the tables associated with our classes.
# Base.metadata.create_all(engine)
# # Create a Session Object to Connect to DB
# # ----------------------------------
# session = Session(bind=engine)

In [ ]:
# Use this to clear out the db
# ----------------------------------
# Session.rollback(self)
# Base.metadata.drop_all(engine)
# session.commit()

## List of all players
find specific player

In [25]:
## NEED
from nba_api.stats.static import players
# get_players returns a list of dictionaries, each representing a player.
nba_players = players.get_players()
print('Number of players fetched: {}'.format(len(nba_players)))
nba_players[:10]
all_players = pd.DataFrame(nba_players)
all_players = all_players.copy()
all_players

Number of players fetched: 4501


,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False
...,...,...,...,...,...
4496,1627790,Ante Zizic,Ante,Zizic,True
4497,78647,Jim Zoet,Jim,Zoet,False
4498,78648,Bill Zopf,Bill,Zopf,False
4499,1627826,Ivica Zubac,Ivica,Zubac,True


In [33]:
all_players_clean = all_players.rename(columns={"id": "PLAYER_ID", 'full_name': 'PLAYER_NAME'})
all_players_clean = all_players_clean.drop(columns=['first_name','last_name','is_active'])
all_players_clean

,PLAYER_ID,PLAYER_NAME
0,76001,Alaa Abdelnaby
1,76002,Zaid Abdul-Aziz
2,76003,Kareem Abdul-Jabbar
3,51,Mahmoud Abdul-Rauf
4,1505,Tariq Abdul-Wahad
...,...,...
4496,1627790,Ante Zizic
4497,78647,Jim Zoet
4498,78648,Bill Zopf
4499,1627826,Ivica Zubac


### Find specific player

In [ ]:
## loop to find player
player_name = [player for player in nba_players
                   if player['full_name'] == 'Tim Duncan'][0]
player_name

## Get Career Stats by player id number
Get player_id number from get_players above

#### Key: 

* GP: Games Played
* MIN: Minutes Played
* FGM: Field Goals Made
* FGA: Field Goals Attempted
* FG%: Field Goal Percentage
* 3PM: 3 Point Field Goals Made
* 3PA: 3 Point Field Goals Attempted
* 3P%: 3 Point Field Goals Percentage
* FTM: Free Throws Made
* FTA: Free Throws Attempted
* FT%: Free Throw Percentage
* OREB: Offensive Rebounds
* DREB: Defensive Rebounds
* REB: Rebounds
* AST: Assists
* TOV: Turnovers
* STL: Steals
* BLK: Blocks
* PF: Personal Fouls
* DD2: Double Doubles
* TD3: Trible Doubles
* PTS: Points
* YIL: Year in League




example

In [10]:
# Display all columns
pd.set_option('display.max_columns', 500)
# Anthony Davis
career = playercareerstats.PlayerCareerStats(player_id='76001')
career_df = career.get_data_frames()[0]

career_dict = career_df.to_dict('records')
first_5_years = career_df[0:5]
first_5_years

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,76001,1990-91,00,1610612757,POR,23.0,43,0,290.0,55,116,0.474,0,0,0.0,25,44,0.568,27,62,89,12,4,12,22,39,135
1,76001,1991-92,00,1610612757,POR,24.0,71,1,934.0,178,361,0.493,0,0,0.0,76,101,0.752,81,179,260,30,25,16,66,132,432
2,76001,1992-93,00,1610612749,MIL,25.0,12,0,159.0,26,56,0.464,0,1,0.0,12,16,0.750,12,25,37,10,6,4,13,24,64
3,76001,1992-93,00,1610612738,BOS,25.0,63,52,1152.0,219,417,0.525,0,0,0.0,76,100,0.760,114,186,300,17,19,22,84,165,514
4,76001,1992-93,00,0,TOT,25.0,75,52,1311.0,245,473,0.518,0,1,0.0,88,116,0.759,126,211,337,27,25,26,97,189,578


In [ ]:
# main_df = pd.DataFrame()
# yil = 1

# def addYil(first_5_years, yil):
#     ## List all season_id column values found
#     season_ids = list(first_5_years['SEASON_ID'].values)
    
#     ## Create list to add to dataframe
#     ## Scans values in SEASON_ID. If change in year is found, adds 1 int to new column 'YIL' (Years in League). 
#     ## We are scanning for 5, 8, and 10 year milestones.
#     ## Changes are scanned due to players under milestones. New rows will add duplicate SEASON_ID if the player changed teams during the season.
#     previous = ''
#     new_yills_add = []
#     for i, value in enumerate(season_ids):
#         print(type(value))
        
#         if value != previous:
#             new_yills_add.append(yil)
#             yil+=1
#     #         print(i)        
#         else:
#             new_yills_add.append(yil - 1)
#             continue

#         previous = season_ids[i]
#     first_5_years_yil = first_5_years.assign(YIL=new_yills_add)  

#     return first_5_years_yil
# first_5_years_yil = addYil(first_5_years, yil)


# main_df = main_df.append( first_5_years_yil, ignore_index = True)


In [ ]:
# ## List all season_id column values found
# season_ids = list(first_5_years['SEASON_ID'].values)

# ## Create list to add to dataframe
# ## Scans values in SEASON_ID. If change in year is found, adds 1 int to new column 'YIL' (Years in League). 
# ## We are scanning for 5, 8, and 10 year milestones.
# ## Changes are scanned due to players under milestones. New rows will add duplicate SEASON_ID if the player changed teams during the season.

# yil = 1
# new_yills_add = []
# previous = ''
# for i, value in enumerate(season_ids):
    
#     if value != previous:
#         new_yills_add.append(yil)
#         yil+=1
# #         print(i)        
#     else:
#         new_yills_add.append(yil - 1)
#         continue
    
#     previous = season_ids[i]
    
# # New dataframe with YIL column
# first_5_years_yil = first_5_years.assign(YIL=new_yills_add)  
# first_5_years_yil

In [ ]:
career_df.columns

-----------

#### PySpark databases 
testing

In [ ]:
# import os
# import findspark
# findspark.init()

In [ ]:
# # Start Spark session
# from pyspark.sql import SparkSession
# from pyspark import SparkFiles
# spark = SparkSession.builder.appName("nbaStats").getOrCreate()

In [ ]:
# with open("sample.json", "w") as outfile:  
#     json.dump(career_dict, outfile) 

In [ ]:
# sc = spark.sparkContext
# path = "./sample.json"
# players_json_spark = spark.read.json(path)

In [ ]:
# players_json_spark.printSchema()

In [ ]:
# Creates a temporary view using the DataFrame
# players_json_spark.createOrReplaceTempView("people")

In [ ]:
# players_json_spark.show()

In [ ]:
# # playersDF_spark = spark.range(3).toDF("myCol")
# newRow = players_json_spark
# appended = players_json_spark.union(newRow)
# display(appended)

In [ ]:
# appended.count()

In [ ]:
# def customFunction(row):

#    return (row)

# sample2 = appended.rdd.map(customFunction)
# sample2

In [ ]:
# for ids in all_players_ids: 
#     player_to_pass = ids

In [ ]:
# for f in appended.collect(): 
#         print (f)

In [ ]:
# playersDF_spark.withColumn('newprice', dataframe]).show()

In [ ]:
# otherPeople = spark.read.json(path)
# otherPeople.show()
# otherPeople.select(otherPeople["AST"]).show()

In [ ]:
# Create DataFrame manually
# dataframe = spark.createDataFrame(path, schema=main_df_columns)
# dataframe.show()

------------

## Get first 5 years of each player id.

In [11]:
all_players_ids = list(all_players['id'].values.astype(str))
len(all_players_ids)

4501

In [12]:
all_players_col = ['PLAYER_ID', 'SEASON_ID', 'LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'PLAYER_AGE', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'YIL']
len(all_players_col)

28

In [13]:
pbar = tqdm(total=len(all_players_ids))
main_df = pd.DataFrame()
yil = 1

def addYil(first_5_years, yil):
    ## List all season_id column values found
    season_ids = list(first_5_years['SEASON_ID'].values)
    
    ## Create list to add to dataframe
    ## Scans values in SEASON_ID. If change in year is found, adds 1 int to new column 'YIL' (Years in League). 
    ## We are scanning for 5, 8, and 10 year milestones.
    ## Changes are scanned due to players under milestones. New rows will add duplicate SEASON_ID if the player changed teams during the season.
    previous = ''
    new_yills_add = []
    
    for i, value in enumerate(season_ids):
#         print(type(value))
        if value != previous:
            new_yills_add.append(yil)
            yil+=1       
        else:
            new_yills_add.append(yil - 1)
            continue

        previous = season_ids[i]
        
    first_5_years_yil = first_5_years.assign(YIL=new_yills_add)  
    return first_5_years_yil


for ids in all_players_ids: 
    player_to_pass = ids

    career = playercareerstats.PlayerCareerStats(player_id=player_to_pass, headers=headers, timeout=100)
    career_df = career.get_data_frames()[0]
    career_df['YIL'] = 0
    first_5_years = career_df[0:5]
    first_5_years_yil = addYil(first_5_years, yil)

    main_df = main_df.append( first_5_years_yil, ignore_index = True)
    time.sleep(1)
    pbar.update(1)
    
pbar.close()



  1%|█▏                                                                            | 65/4501 [01:26<1:56:41,  1.58s/it]


  3%|██▏                                                                          | 131/4501 [03:06<1:50:20,  1.51s/it]


  4%|███▎                                                                         | 197/4501 [04:47<1:48:16,  1.51s/it]


  6%|████▍                                                                        | 263/4501 [06:27<1:46:06,  1.50s/it]


  7%|█████▋                                                                       | 329/4501 [08:07<1:45:26,  1.52s/it]


...

 95%|████████████████████████████████████████████████████████████████████████▍   | 4289/4501 [1:50:23<05:21,  1.52s/it]


 97%|█████████████████████████████████████████████████████████████████████████▌  | 4355/4501 [1:52:05<03:44,  1.54s/it]


 98%|██████████████████████████████████████████████████████████████████████████▋ | 4421/4501 [1:53:46<02:02,  1.53s/it]


100%|███████████████████████████████████████████████████████████████████████████▊| 4487/4501 [1:55:27<00:21,  1.52s/it]


100%|████████████████████████████████████████████████████████████████████████████| 4501/4501 [1:55:48<00:00,  1.54s/it]


In [73]:
main_df_pass = main_df.copy()

In [80]:
#  Player positions
player_positions = pd.read_csv('../../datasets/nba_positions.csv') 
player_positions

,POSITION,PLAYER_NAME
0,PG,Stephen Curry
1,PG,Chris Paul
2,PG,Russell Westbrook
3,PG,John Wall
4,SF,Kevin Durant
...,...,...
835,PF,Tyrone Hill
836,PF,Clifford Robinson
837,SF,Jamal Mashburn
838,PG,Steve Nash


In [81]:
five_year_all_players = main_df_pass.sort_values(by='SEASON_ID', ascending=False).copy()
five_year_all_players = five_year_all_players.reset_index(drop=True)
five_year_all_players = five_year_all_players.join(all_players_clean.set_index('PLAYER_ID'), on='PLAYER_ID')
# five_year_all_players = players_with_names.join(player_positions.set_index('PLAYER_NAME'), on='PLAYER_NAME')
five_year_all_players

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,YIL,PLAYER_NAME
0,1628605,2019-20,00,1610612763,MEM,26.0,2,0,13,4,9,0.444,2,3,0.667,2,2,1,0,1,1,0,0,0,0,1,12,2.0,Dusty Hannahs
1,1629658,2019-20,00,1610612757,POR,21.0,13,0,103,15,32,0.469,0,3,0,8,13,0.615,14,18,32,4,5,0,4,12,38,1.0,Jaylen Hoard
2,1629682,2019-20,00,1610612738,BOS,22.0,11,1,119,14,49,0.286,4,24,0.167,8,8,1,0,12,12,16,10,2,15,13,40,1.0,Tremont Waters
3,1628407,2019-20,00,1610612766,CHA,24.0,39,11,687,85,244,0.348,19,67,0.284,35,53,0.66,15,85,100,52,23,2,37,52,224,3.0,Dwayne Bacon
4,1629644,2019-20,00,1610612748,MIA,21.0,5,0,26,3,5,0.6,0,1,0,1,2,0.5,1,4,5,1,2,1,1,2,7,1.0,KZ Okpala
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15113,78019,1946-47,00,0,TOT,27.0,60,None,NaN,119,390,0.305,None,None,None,144,225,0.64,None,None,NaN,36,None,None,None,172,382,1.0,Hank Rosenstein
15114,76720,1946-47,00,0,TOT,24.0,60,None,NaN,70,362,0.193,None,None,None,81,130,0.623,None,None,NaN,35,None,None,None,153,221,1.0,Bob Fitzgerald
15115,77503,1946-47,00,1610610035,HUS,25.0,60,None,NaN,236,838,0.282,None,None,None,177,288,0.615,None,None,NaN,59,None,None,None,184,649,1.0,Mike McCarron
15116,76720,1946-47,00,1610610035,HUS,24.0,31,None,0,47,241,0.195,None,None,None,45,70,0.643,None,None,0,26,None,None,None,86,139,1.0,Bob Fitzgerald


In [82]:
five_year_all_players_to_json = five_year_all_players.to_json(orient='records')

In [83]:
# SAVE: Player_position
with open(f'./_players_all_data.json', 'w') as fp:
    json.dump(five_year_all_players_to_json, fp)

-------------

# Open Saved JSON

In [91]:
with open('./_players_all_data.json') as json_file:
    data = json.load(json_file)

In [92]:
five_year_all_players = pd.read_json(data)
twenty_years_all_players = five_year_all_players[five_year_all_players['SEASON_ID'] > '2000-99']
twenty_years_all_players

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,YIL,PLAYER_NAME
0,1628605,2019-20,0,1610612763,MEM,26,2,0.0,13.0,4,9,0.444,2.0,3.0,0.667,2,2,1.000,0.0,1.0,1.0,0,0.0,0.0,0.0,1,12,2,Dusty Hannahs
1,1629658,2019-20,0,1610612757,POR,21,13,0.0,103.0,15,32,0.469,0.0,3.0,0.000,8,13,0.615,14.0,18.0,32.0,4,5.0,0.0,4.0,12,38,1,Jaylen Hoard
2,1629682,2019-20,0,1610612738,BOS,22,11,1.0,119.0,14,49,0.286,4.0,24.0,0.167,8,8,1.000,0.0,12.0,12.0,16,10.0,2.0,15.0,13,40,1,Tremont Waters
3,1628407,2019-20,0,1610612766,CHA,24,39,11.0,687.0,85,244,0.348,19.0,67.0,0.284,35,53,0.660,15.0,85.0,100.0,52,23.0,2.0,37.0,52,224,3,Dwayne Bacon
4,1629644,2019-20,0,1610612748,MIA,21,5,0.0,26.0,3,5,0.600,0.0,1.0,0.000,1,2,0.500,1.0,4.0,5.0,1,2.0,1.0,1.0,2,7,1,KZ Okpala
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5482,2351,2001-02,0,1610612745,HOU,25,65,13.0,1077.0,135,341,0.396,37.0,126.0,0.294,82,105,0.781,47.0,75.0,122.0,40,25.0,9.0,49.0,64,389,1,Oscar Torres
5483,1503,2001-02,0,1610612753,ORL,23,76,76.0,2912.0,715,1586,0.451,103.0,283.0,0.364,415,555,0.748,150.0,447.0,597.0,400,119.0,73.0,189.0,139,1948,5,Tracy McGrady
5484,2091,2001-02,0,1610612756,PHX,26,3,0.0,24.0,4,8,0.500,0.0,0.0,0.000,0,0,0.000,0.0,7.0,7.0,2,0.0,1.0,2.0,4,8,2,Daniel Santiago
5485,2031,2001-02,0,1610612763,MEM,22,68,14.0,1805.0,293,611,0.480,0.0,3.0,0.000,217,305,0.711,161.0,269.0,430.0,50,53.0,113.0,122.0,178,803,2,Stromile Swift


#### Noteable Errors
NaN is applied to position if the player position is not found with player salaries. Will fill NaN afterwards.

In [93]:
twenty_years_all_players = twenty_years_all_players.join(player_positions.set_index('PLAYER_NAME'), on='PLAYER_NAME')
twenty_years_all_players.copy()

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,YIL,PLAYER_NAME,POSITION
0,1628605,2019-20,0,1610612763,MEM,26,2,0.0,13.0,4,9,0.444,2.0,3.0,0.667,2,2,1.000,0.0,1.0,1.0,0,0.0,0.0,0.0,1,12,2,Dusty Hannahs,NaN
1,1629658,2019-20,0,1610612757,POR,21,13,0.0,103.0,15,32,0.469,0.0,3.0,0.000,8,13,0.615,14.0,18.0,32.0,4,5.0,0.0,4.0,12,38,1,Jaylen Hoard,NaN
2,1629682,2019-20,0,1610612738,BOS,22,11,1.0,119.0,14,49,0.286,4.0,24.0,0.167,8,8,1.000,0.0,12.0,12.0,16,10.0,2.0,15.0,13,40,1,Tremont Waters,NaN
3,1628407,2019-20,0,1610612766,CHA,24,39,11.0,687.0,85,244,0.348,19.0,67.0,0.284,35,53,0.660,15.0,85.0,100.0,52,23.0,2.0,37.0,52,224,3,Dwayne Bacon,NaN
4,1629644,2019-20,0,1610612748,MIA,21,5,0.0,26.0,3,5,0.600,0.0,1.0,0.000,1,2,0.500,1.0,4.0,5.0,1,2.0,1.0,1.0,2,7,1,KZ Okpala,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5485,2031,2001-02,0,1610612763,MEM,22,68,14.0,1805.0,293,611,0.480,0.0,3.0,0.000,217,305,0.711,161.0,269.0,430.0,50,53.0,113.0,122.0,178,803,2,Stromile Swift,NaN
5486,1887,2001-02,0,1610612750,MIN,25,82,82.0,3121.0,609,1200,0.507,87.0,191.0,0.455,226,272,0.831,120.0,271.0,391.0,257,66.0,21.0,181.0,188,1531,3,Wally Szczerbiak,SG
5486,1887,2001-02,0,1610612750,MIN,25,82,82.0,3121.0,609,1200,0.507,87.0,191.0,0.455,226,272,0.831,120.0,271.0,391.0,257,66.0,21.0,181.0,188,1531,3,Wally Szczerbiak,SG
5486,1887,2001-02,0,1610612750,MIN,25,82,82.0,3121.0,609,1200,0.507,87.0,191.0,0.455,226,272,0.831,120.0,271.0,391.0,257,66.0,21.0,181.0,188,1531,3,Wally Szczerbiak,SG


In [110]:
import plotly.express as px


fig = px.scatter(twenty_years_all_players, x="YIL", y="PTS", log_x=True, hover_name="PLAYER_ID", hover_data=["PTS"])

fig.show()

In [ ]:
# five_year_all_players.set_index('SEASON_ID')

In [ ]:
## Save all players data to sqlite db 'all_players'
# all_players.to_sql('all_players', con=engine)

## Retreive all players from sqlite db
# engine.execute("SELECT * FROM all_players").fetchall()

https://plotly.com/python/polar-chart/

In [111]:
import plotly.express as px
df = px.data.wind()
fig = px.line_polar(twenty_years_all_players, r="PTS", theta="POSITION", color="YIL", line_close=True,
                    color_discrete_sequence=px.colors.sequential.Plasma_r,
                    template="plotly_dark",)
fig.show()

In [109]:
df = px.data.wind()
df

,direction,strength,frequency
0,N,0-1,0.5
1,NNE,0-1,0.6
2,NE,0-1,0.5
3,ENE,0-1,0.4
4,E,0-1,0.4
...,...,...,...
123,WSW,6+,0.1
124,W,6+,0.9
125,WNW,6+,2.2
126,NW,6+,1.5
